In [21]:
import os
import json
import pandas as pd
import traceback

In [22]:
from langchain.chat_models import ChatOpenAI

In [23]:
# load the value which is written in .env file
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
api_key = os.getenv("OPENAI_API_KEY")

In [25]:

llm = ChatOpenAI(openai_api_key = api_key, model_name = 'gpt-3.5-turbo', temperature=0.5)

In [26]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SequentialChain 
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
import PyPDF2

In [27]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [28]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE JSON below and use it as a guide. \
Ensure to make {number} MCQS
### RESPONSE_JSON
{response_json}
"""



In [29]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [30]:
quiz_chain = LLMChain(llm = llm, prompt= quiz_generation_prompt, output_key= "quiz", verbose = True)

In [31]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [32]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template= TEMPLATE2)

In [33]:
review_chain = LLMChain(llm = llm, prompt= quiz_evaluation_prompt, output_key="review", verbose= True)

In [34]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [35]:
file_path = "D:\Coding file\Generative_AI\code\MCQ_GENERATOR\data.txt"

In [36]:
with open (file_path,'r') as file:
    TEXT = file.read()

In [37]:
TEXT

'Romanticism was an artistic, literary, and intellectual movement that originated in Europe in the late 18th century and peaked in the first half of the 19th century. It was characterized by a deep appreciation of the beauty and power of nature, an emphasis on emotion and individualism, and a fascination with the past, particularly the medieval era and the sublime. Romanticism emerged as a reaction against the Industrial Revolution and the scientific rationalization of nature, which were hallmarks of the Enlightenment.\n\nIn literature, Romanticism was marked by the works of poets such as William Wordsworth, Samuel Taylor Coleridge, Lord Byron, Percy Bysshe Shelley, and John Keats in England, and Johann Wolfgang von Goethe in Germany. These writers often explored themes of beauty, nature, love, and the struggles of the human spirit. They celebrated the individual’s connection to the natural world and emphasized the importance of imagination and personal experience over societal norms a

In [39]:
# dictionary to json
response_json = json.dumps(RESPONSE_JSON)

In [41]:
NUMBER = 5
SUBJECT ="Romanticism"
TONE = "simple"

In [42]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text" : TEXT,
            "number" : NUMBER,
            "subject" : SUBJECT,
            "tone" : TONE,
            "response_json" : response_json
        }
    )

c:\Users\hp\.conda\envs\GENAI\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Romanticism was an artistic, literary, and intellectual movement that originated in Europe in the late 18th century and peaked in the first half of the 19th century. It was characterized by a deep appreciation of the beauty and power of nature, an emphasis on emotion and individualism, and a fascination with the past, particularly the medieval era and the sublime. Romanticism emerged as a reaction against the Industrial Revolution and the scientific rationalization of nature, which were hallmarks of the Enlightenment.

In literature, Romanticism was marked by the works of poets such as William Wordsworth, Samuel Taylor Coleridge, Lord Byron, Percy Bysshe Shelley, and John Keats in England, and Johann Wolfgang von Goethe in Germany. These writers often explored themes of beauty, nature, love, and the struggles of the human spirit. They celebrated the individual’s connection to th

In [45]:
print(response)

{'text': 'Romanticism was an artistic, literary, and intellectual movement that originated in Europe in the late 18th century and peaked in the first half of the 19th century. It was characterized by a deep appreciation of the beauty and power of nature, an emphasis on emotion and individualism, and a fascination with the past, particularly the medieval era and the sublime. Romanticism emerged as a reaction against the Industrial Revolution and the scientific rationalization of nature, which were hallmarks of the Enlightenment.\n\nIn literature, Romanticism was marked by the works of poets such as William Wordsworth, Samuel Taylor Coleridge, Lord Byron, Percy Bysshe Shelley, and John Keats in England, and Johann Wolfgang von Goethe in Germany. These writers often explored themes of beauty, nature, love, and the struggles of the human spirit. They celebrated the individual’s connection to the natural world and emphasized the importance of imagination and personal experience over societa